# Preparació de dades

In [1]:
import pandas as pd
import numpy as np
import h5py
import os

from sklearn.preprocessing import MinMaxScaler, StandardScaler

## Lectura de dades de l'ABC Database

In [2]:
# Dades auxiliars (paràmetres planetaris i estel·lars)
df_aux = pd.read_csv("/kaggle/input/abc-database/Level2Data/AuxillaryTable.csv")

# Espectres atmosfèrics (variables predictores)
spectral_data = h5py.File("/kaggle/input/abc-database/Level2Data/SpectralData.hdf5", 'r')

# Paràmetres atmosfèrics (variables objectiu)
FM_params = pd.read_csv("/kaggle/input/abc-database/Level2Data/Ground Truth Package/FM_Parameter_Table.csv")

### Estructura de les dades

In [3]:
print("Nombre total de dades auxiliars: ", df_aux.shape)
print("Nombre total d'espectres:", len(spectral_data.keys()))
print("Nombre total de FM_params rows: ", FM_params.shape)

# Exemple de les primeres keys: són el planet_ID
#list(spectral_data.keys())[:5]

# Per cada planeta, tenim:
#list(spectral_data['Planet_0'].keys())

Nombre total de dades auxiliars:  (91392, 10)
Nombre total d'espectres: 91392
Nombre total de FM_params rows:  (91392, 7)


Comprovem si els registres estan alineats entre conjunts de dades.

Veiem que no: els espectres atmosfèrics (spectral_data) estan desordenats, per tant, ens interessa alinear-los amb els altres dos conjunts de dades (FM_params i df_aux).

In [4]:
# Comparem estructura de la columna ID
# FM_params i df_aux coincideixen. spectral_data està desordenat
print(df_aux['planet_ID'].to_numpy()[:5])
print(FM_params['planet_ID'].to_numpy()[:5])
print(list(spectral_data.keys())[:5])

[0 1 2 3 4]
[0 1 2 3 4]
['Planet_0', 'Planet_1', 'Planet_10', 'Planet_100', 'Planet_1000']


## Preparar dades X (espectres)

Ordenem els espectres perquè coincideixin amb les dades auxiliars i els paràmetres atmosfèrics.

In [5]:
# Noms de les variables espectrals
wl_values = np.round(spectral_data['Planet_0']['instrument_wlgrid'][:], 2)
wl_names = ['wl_' + str(x) for x in wl_values ]


In [6]:
# Funció per posar tots els espectres (data_file) en una matriu,
# ordenats igual que en FM_params o df_aux (aux_file)
def to_observed_matrix(data_file,aux_file):
    # Els espectres estan desordenats. Els alineem amb les dades auxiliars.
    id_order = aux_file['planet_ID'].to_numpy() # IDs de les dades auxiliars --> array([    0,     1,     2, ..., 91389, 91390, 91391])
    
    # Inicialitzem array per posar-hi els espectres
    num = len(data_file.keys()) # Nombre d'espectres
    observed_spectrum = np.zeros((num, 52)) 

    for idx, x in enumerate(id_order):
        current_planet_id = f'Planet_{x}'
        observed_spectrum[idx,:] = data_file[current_planet_id]['instrument_spectrum'][:]
        
    return observed_spectrum

In [7]:
# Creem la matriu d'espectres observats (X)
observed_matrix = to_observed_matrix(spectral_data, FM_params)
print("Dimensions de la matriu d'espectres: ", observed_matrix.shape)

# Mostra d'un espectre raw:
print("Mostra d'un espectre raw: \n", observed_matrix[0,:])

Dimensions de la matriu d'espectres:  (91392, 52)
Mostra d'un espectre raw: 
 [0.00167053 0.00145612 0.00146214 0.00143407 0.00138982 0.00134926
 0.00131032 0.00128092 0.00131345 0.00135396 0.00149923 0.00156278
 0.00163719 0.00167336 0.00168792 0.001731   0.00174372 0.00170247
 0.0017284  0.00165156 0.00150569 0.00136781 0.00130069 0.00125444
 0.00122416 0.00121673 0.00125308 0.00131122 0.00136757 0.00138232
 0.00142169 0.00146799 0.00151336 0.00154108 0.00151537 0.00150134
 0.00147106 0.00138322 0.00130667 0.00124601 0.00118818 0.00115435
 0.00115514 0.00122624 0.00133425 0.00117747 0.00125106 0.00112899
 0.00120413 0.00108887 0.00099446 0.00103997]


A continuació, escalem cada espectre entre 0 i 100, per maximitzar la senyal de les seves característiques.

In [8]:
# Escalem cada espectre entre 0 i 100.
scaler = MinMaxScaler(feature_range = (0,100))
observed_matrix_scaled = scaler.fit_transform(observed_matrix.T).T
observed_matrix_scaled = np.round(observed_matrix_scaled,3)

# Mostra d'un espectre escalat
print("Mostra d'un espectre raw: \n", observed_matrix_scaled[0,:])

Mostra d'un espectre raw: 
 [ 90.231  61.615  62.418  58.672  52.766  47.353  42.156  38.232  42.573
  47.98   67.369  75.851  85.781  90.609  92.552  98.302 100.     94.495
  97.955  87.7    68.231  49.829  40.87   34.697  30.656  29.665  34.517
  42.276  49.797  51.766  57.02   63.199  69.255  72.954  69.523  67.651
  63.609  51.885  41.669  33.573  25.855  21.34   21.445  30.934  45.349
  24.425  34.247  17.954  27.984  12.6     0.      6.073]


### Reducció de la memòria de X

Com que es tracta d'un conjunt de dades molt gran, optimitzem l'ús de memòria:

In [9]:
# Funció que optimitza l'ús de memòria per grans conjunts de dades
# https://www.kaggle.com/general/156268
# https://www.kaggle.com/code/rinnqd/reduce-memory-usage/notebook
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [10]:
df = pd.DataFrame(observed_matrix_scaled)
#df.memory_usage().sum() / 1024**2 #MB
df_reduced = reduce_mem_usage(df)
observed_matrix_scaled_reduced = df_reduced.values


Memory usage after optimization is: 9.06 MB
Decreased by 75.0%


## Preparar dades Y (paràmetres atmosfèrics)

In [11]:
# Creem la matriu de variables resposta (ja està ben ordenada)
params_matrix = FM_params.drop(['planet_ID'], axis=1).to_numpy()
params_matrix.shape

#scaler_Y = StandardScaler()
#params_matrix_scaled = scaler_Y.fit_transform(params_matrix)

(91392, 6)

In [12]:
# Noms de les variables resposta
param_names = FM_params.drop(['planet_ID'], axis=1).columns.tolist()
param_names

['planet_temp', 'log_H2O', 'log_CO2', 'log_CH4', 'log_CO', 'log_NH3']

## Guardar les dades

In [13]:
# Guardar un array (https://www.kaggle.com/code/febymelania/save-and-load-array)

# Dades X 
np.save('/kaggle/working/X_names', wl_names)
np.save('/kaggle/working/X_wl_values', wl_values)
np.save('/kaggle/working/X', observed_matrix)
np.save('/kaggle/working/X_minMaxScaled', observed_matrix_scaled) # scaled
np.save('/kaggle/working/X_minMaxScaled_opt', observed_matrix_scaled_reduced) # scaled + memory-optimised

# Dades Y 
np.save('/kaggle/working/Y_FM_names', param_names)
np.save('/kaggle/working/Y_FM', params_matrix)
#np.save('/kaggle/working/Y_FM_standScaled', params_matrix_scaled)



In [14]:
# Carregar dades guardades
# np.load('/kaggle/working/X_minMaxScaled_opt.npy') 

# Eliminar fitxers
# os.remove("/kaggle/working/X_data_names.npy")
# os.remove("/kaggle/working/X_data_minMaxScaled.npy")
# os.remove("/kaggle/working/Y_data_names.npy")
# os.remove("/kaggle/working/Y_data_standScaled.npy")